# Computing RSAM data
Real-time Seismic Amplitude Measurement (RSAM) was a real-time computer hardware & software system developed by Tom Murray to monitor the St. Helens eruption c. 1985. Since then, any downsampled seismic amplitude data generally gets called "RSAM data" in volcano-seismology circles. It is simple to calculate. Just take the average amplitude of seismic data at regular time intervals. The original RSAM system did it at 2.56-second, 1-minute and 10-minute intervals. So we want to write a function that we can use to do this at any interval.

### Read seismic data
In this case, we read 26 hours of seismic data leading up to the 2019/12/09 01:11 UTC eruption at White Island volcano, New Zealand

In [1]:
# Standard imports
import os
import numpy as np
import matplotlib.pyplot as plt
import obspy

# These 2 lines give us access to the GeoNET (New Zealand) digital seismic network data
#from obspy.clients.fdsn import Client as FDSN_Client
client_geonet = obspy.clients.fdsn.Client("GEONET")

# Define the data that we want
start_time = obspy.UTCDateTime("2019-12-08T00:00:00.000")
hours_of_data = 26
end_time = start_time + hours_of_data * 60 * 60
net = "NZ" # New Zealand national network
stations = ["WIZ", "WSRZ"] # two stations on White Island
chan = "HHZ" # Vertical high-rate channel (100 samples per second)

# Loading data from the GeoNET client is slow. So when we read it, we will save it to a file. 
# Then if we run this program again, we just need to read the file.
fname = 'WhiteIslandFinal26Hours_VEL.mseed'
if os.path.exists(fname):
    # the file already exists, so read it
    print("%s exists. Reading." % fname)
    VEL = obspy.read(fname)
else:
    st = obspy.Stream() # this will hold our raw seismic data
    for station in stations:   
        this_st = client_geonet.get_waveforms(net, station,"*", chan, start_time, end_time, attach_response=True)
        for tr in this_st:
            st = st.append(tr)
    
    # remove instrument response
    VEL = st.copy()
    VEL.detrend(type='linear') # remove a linear trend
    VEL.filter('bandpass', freqmin=0.1, freqmax=25.0, zerophase=True, corners=2) # filter from 0.1 - 25 Hz
    VEL.remove_response(output='VEL')
    VEL.write(fname)
print(VEL)
print('Done.')
VEL.plot();

AttributeError: module 'obspy' has no attribute 'clients'

### Function to compute RSAM data

In [ ]:
def compute_rsam(st, rsam_sampling_interval):
    # Inputs:
    #    st: a Stream object
    #    rsam_sampling_interval: sampling interval for RSAM data (in seconds)
    # Outputs:
    #    rsam_stream: a downsampled Stream object. 
    
    rsam_stream = st.copy() # we copy the input as it is passed-by-reference and would otherwise be changed in place
    
    for tr in rsam_stream: # Loop over each trace (seismogram)
        x = tr.data # get the data
        
        # now we want to reshape the data vector into an array, so we can take advantage of np.mean()
        s = np.size(x) # find the size of the data vector
        print('%s: size %d' % (tr.id, s))
        nc = int(tr.stats.sampling_rate * rsam_sampling_interval) # number of columns
        nr = int(s / nc) # number of rows
        x = x[0:nr*nc] # cut off any trailing samples
        y = x.reshape((nr, nc))
        print('%s: reshaped to %d x %d array (%d samples)' % (tr.id, nr, nc, nr * nc))
        tr.data = np.mean(abs(y),axis=1) # compute mean for each row (this is vectorised; faster than a for loop)
        tr.stats.sampling_rate = 1.0 / rsam_sampling_interval # update the sampling rate
        print(tr.stats)
        print("")
        
    return rsam_stream

In [ ]:
def plot_rsam(st):
    hf = st.plot(handle=True); # standard ObsPy plot
    
    # change the y-axis so it starts at 0
    allAxes = hf.get_axes()
    for ax in allAxes:
        ylims = ax.get_ylim()
        ax.set_ylim([0, ylims[1]])  

In [ ]:
# 2.56-second RSAM
rsam_highrate = compute_rsam(VEL, 2.56)
plot_rsam(rsam_highrate);

In [ ]:
# 1-minute RSAM
rsam1m = compute_rsam(VEL, 60)
plot_rsam(rsam1m);

In [ ]:
# 10-minute RSAM
rsam10m = compute_rsam(VEL, 10*60)
plot_rsam(rsam10m);

### Write RSAM data to file?
Simple CSV file of date/time + value?

In [ ]:
# Write a new file
f = open("HelloWorld.txt", "w") # use "a" to append to an existing file
f.write("Now the file has more content!")
f.close()

In [ ]:
def write_rsam(rsamfile, tr):
    f = open(rsamfile, "w")
    dt = tr.times(type="UTCDateTime")
    f.write('year, month, day, hour, minute, second\n')
    for i in range(len(tr.data)):
        f.write("%s, %e\n" % (dt[i].strftime('%Y, %m, %d, %H, %M, %S'), tr.data[i]) )
    f.close()

In [ ]:
write_rsam('rsamfile.csv',rsam10m[0])

# Redoubt volcano eruption, March 2009
Redoubt volcano in Alaska erupted (at least) 19 times from March-April, 2009. The first magmatic explosion was preceded by an earthquake swarm. Events got closer and closer together, merging into tremor. 

In [ ]:
import obspy
# Data from Redoubt 2009?
client_iris = obspy.clients.fdsn.Client("IRIS")

# Define the data that we want
start_time = obspy.UTCDateTime("2009-03-22T00:00:00.000")
hours_of_data = 32
end_time = start_time + hours_of_data * 60 * 60
net = "AV" 
stations = ["RSO", "REF"] # two stations on Redoubt Island
chan = "EHZ" # Vertical channel 

# Loading data from the GeoNET client is slow. So when we read it, we will save it to a file. 
# Then if we run this program again, we just need to read the file.
fname = 'Redoubt20090322_VEL.mseed'

In [ ]:
if os.path.exists(fname):
    # the file already exists, so read it
    print("%s exists. Reading." % fname)
    VEL = obspy.read(fname)
else:
    st = obspy.Stream() # this will hold our raw seismic data
    for station in stations:   
        this_st = client_iris.get_waveforms(net, station,"*", chan, start_time, end_time, attach_response=True)
        this_st.merge(method=1, fill_value=0)
        for tr in this_st:
            st = st.append(tr)
    
    # remove instrument response
    VEL = st.copy()
    VEL.detrend(type='linear') # remove a linear trend
    VEL.filter('bandpass', freqmin=0.5, freqmax=25.0, zerophase=True, corners=2) # filter from 0.5 - 25 Hz
    VEL.remove_response(output='VEL')
    VEL.write(fname)
print(VEL)
print('Done.')
VEL.plot();

In [ ]:
#VEL.merge()
print(VEL)

In [ ]:
for tr in VEL:
    tr.plot(type="dayplot")

In [ ]:
# 1-minute RSAM
rsam1m = compute_rsam(VEL, 10*60)
plot_rsam(rsam1m);

In [ ]:
# write these RSAM data to file
write_rsam('rsamfile.csv',rsam1m[0])